# Experimento 1 - Sin Restricciones Adicionales

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append("..")

import pickle
import random

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# Use latex
plt.rc("text", usetex=True)

## Definición

In [5]:
from experimento import Experimento
from src.generacion import (
    GeneradorInstancias,
    DistribucionIndependiente,
    DistribucionUniforme,
    DistribucionNormal,
)


experimento = Experimento(
    nombre="exp1",
    generador=GeneradorInstancias(
        cantidad_trabajadores=DistribucionUniforme(5, 10),
        cantidad_ordenes=DistribucionUniforme(5, 20),
        parametros_ordenes=DistribucionIndependiente(
            DistribucionNormal(10000, 2000),
            DistribucionUniforme(3, 10),
        ),
    ),
    N=1000,
    seed=42,
)

## Ejecución

In [8]:
resultados = experimento.ejecutar()

exp1: 100%|██████████| 1000/1000 [08:08<00:00,  2.05it/s] 


FileNotFoundError: [Errno 2] No such file or directory: '/home/sponja/Projects/invop-tp/experimentos/datos/exp1/resultados.pkl'

In [9]:
resultados

NameError: name 'resultados' is not defined